This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-model-cookbooks/aloha).

## Aloha Demo

In this notebook we will walk through a simple pipeline deployment to inference on a model. For this example we will be using an open source model that uses an [Aloha CNN LSTM model](https://www.researchgate.net/publication/348920204_Using_Auxiliary_Inputs_in_Deep_Learning_Models_for_Detecting_DGA-based_Domain_Names) for classifying Domain names as being either legitimate or being used for nefarious purposes such as malware distribution.  

For our example, we will perform the following:

* Create a workspace for our work.
* Upload the Aloha model.
* Create a pipeline that can ingest our submitted data, submit it to the model, and export the results
* Run a sample inference through our pipeline by loading a file
* Run a sample inference through our pipeline's URL and store the results in a file.

All sample data and models are available through the [Wallaroo Quick Start Guide Samples repository](https://github.com/WallarooLabs/quickstartguide_samples).

## Open a Connection to Wallaroo

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

In [1]:
import wallaroo
import asyncio 
import datetime
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
import pandas as pd
pd.set_option('display.max_colwidth', None)

# to display dataframe tables
from IPython.display import display

In [2]:
# Client connection from local Wallaroo instance

wl = wallaroo.Client()

Please log into the following URL in a web browser:

	https://sparkly-apple-3026.keycloak.wallaroo.community/auth/realms/master/device?user_code=GHMU-YILH

Login successful!


## Create the Workspace

We will create a workspace to work in and call it the "alohaworkspace", then set it as current workspace environment.  We'll also create our pipeline in advance as `alohapipeline`.  The model name and the model file will be specified for use in later steps.

To allow this tutorial to be run multiple times or by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.

In [4]:
import string
import random

# make a random 4 character prefix
prefix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))
workspace_name = f"{prefix}alohaworkspace"
pipeline_name = f"{prefix}alohapipeline"
model_name = f"{prefix}alohamodel"
model_file_name = './alohacnnlstm.zip'

In [5]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [6]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

aloha_pipeline = get_pipeline(pipeline_name)
aloha_pipeline

{'name': 'voomalohapipeline', 'create_time': datetime.datetime(2023, 6, 26, 19, 33, 33, 632845, tzinfo=tzutc()), 'definition': '[]'}

# Upload the Models

Now we will upload our models.  Note that for this example we are applying the model from a .ZIP file.  The Aloha model is a [protobuf](https://developers.google.com/protocol-buffers) file that has been defined for evaluating web pages, and we will configure it to use data in the `tensorflow` format.

In [7]:
model = wl.upload_model(model_name, model_file_name).configure("tensorflow")

## Deploy a model

Now that we have a model that we want to use we will create a deployment for it. 

We will tell the deployment we are using a tensorflow model and give the deployment name and the configuration we want for the deployment.

In [8]:
aloha_pipeline.add_model_step(model)

{'name': 'voomalohapipeline', 'create_time': datetime.datetime(2023, 6, 26, 19, 33, 33, 632845, tzinfo=tzutc()), 'definition': "[{'ModelInference': {'models': [{'name': 'voomalohamodel', 'version': 'a83f7c41-5b69-4485-92d4-8c6c326db3f6', 'sha': 'd71d9ffc61aaac58c2b1ed70a2db13d1416fb9d3f5b891e5e4e2e97180fe22f8'}]}}]"}

In [9]:
REPLICAS = 4
deployment_config = (wallaroo.DeploymentConfigBuilder()
    .cpus(1)
    .memory("0.5Gi")
    .replica_count(REPLICAS)
    .build())

In [10]:
aloha_pipeline = aloha_pipeline.deploy(deployment_config =deployment_config)

Waiting for deployment - this will take up to 45s .............. ok


We can verify that the pipeline is running and list what models are associated with it.

In [12]:
aloha_pipeline.status()

## Interferences

### Infer 1 row

Now that the pipeline is deployed and our Aloha model is in place, we'll perform a smoke test to verify the pipeline is up and running properly.  We'll use the `infer_from_file` command to load a single encoded URL into the inference engine and print the results back out.

The result should tell us that the tokenized URL is legitimate (0) or fraud (1).  This sample data should return close to 0.

In [ ]:
result = aloha_pipeline.infer_from_file('./data/data_1.df.json')

display(result)

### Batch Inference Example

Now we'll perform a batch inference.  We have the file `./data/data_25k.df.json`, which is a pandas DataFrame file with 25,000 records to analyze.  We'll provide it to the pipeline and perform a sample inference, and provide the first 20 rows.

In [ ]:
%time
batch_result = aloha_pipeline.infer_from_file('./data/data_25k.df.json')
display(batch_result.head(20))

### Parallel Inference Example

This time, let's take the same file and split it into 25,000 **separate** DataFrames, which each indivual row as a single DataFrame.  This is toy data; we're just providing it as an example of how to submit a an inference request for parallel infer.

In [ ]:
test_data = pd.read_json("./data/data_25k.df.json")
test_list = []

for index, row in test_data.iterrows():
    test_list.append(row.to_frame('text_input').reset_index())

Now we'll perform an inference with Parallel Infer through the pipeline.

The pipeline `parallel_infer(tensor_list, timeout, num_parallel, retries)` **asynchronous** method performs an inference as defined by the pipeline steps and takes the following arguments:

* **tensor_list** (*REQUIRED List*): The data submitted to the pipeline for inference as a List of the supported data types:
  * [pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html):  Data submitted as a pandas DataFrame are returned as a pandas DataFrame.  For models that output one column  based on the models outputs.
  * [Apache Arrow](https://arrow.apache.org/) (**Preferred**): Data submitted as an Apache Arrow are returned as an Apache Arrow.
* **timeout** (*OPTIONAL int*): A timeout in seconds before the inference throws an exception.  The default is 15 second per call to accommodate large, complex models.  Note that for a batch inference, this is **per list item** - with 10 inference requests, each would have a default timeout of 15 seconds.
* **num_parallel** (*OPTIONAL int*):  The number of parallel threads used for the submission.  **This should be no more than four times the number of pipeline replicas**.
* **retries** (*OPTIONAL int*):  The number of retries per inference request submitted.

`parallel_infer` is an asynchronous method that returns the Python callback list of tasks. Calling `parallel_infer` should be called with the `await` keyword to retrieve the callback results.

In [14]:
timeout_secs=1200
now = datetime.datetime.now()
##########
parallel_results = await aloha_pipeline.parallel_infer(tensor_list=test_list, timeout=timeout_secs, num_parallel=2*8, retries=3)
##########
total = datetime.datetime.now() - now
print(f"Elapsed_in_parallel = {total.total_seconds()} : {len(parallel_results)}")

## Undeploy Pipeline

When finished with our tests, we will undeploy the pipeline so we have the Kubernetes resources back for other tasks.  Note that if the deployment variable is unchanged aloha_pipeline.deploy() will restart the inference engine in the same configuration as before.

In [ ]:
aloha_pipeline.undeploy()

In [ ]:
time.sleep(1)

In [ ]:
import asyncio

lock = asyncio.Lock()

async def async_task(i):
    print(f"Async {i} task starts")
    await sync_task_wrapper(i)
    await asyncio.sleep(1)
    print(f"Async {i} task finishes")
    
async def sync_task_wrapper(i):
    async with lock:
        await asyncio.get_event_loop().run_in_executor(sync_task(i))

def sync_task(n):
    print(f"Sync task {n} starts")
    # Simulating a blocking operation
    for i in range(100):
        pass
    print(f"Sync task {n} finishes")

async def main():
    print("Main task starts")
    tasks = [
        async_task(i)
        for i in range(5)
    ]
    await asyncio.gather(*tasks, return_exceptions=True)
    await asyncio.sleep(2)  # Simulating other async operations
    print("Main task finishes")


In [ ]:
await main()

In [ ]:
def wrapper_infer():
    asyncio.run(parallel())

In [ ]:
import threading
threads = []
for i in range(3):
    thread = threading.Thread(target=wrapper_main)
    threads.append(thread)
    print("starting new thread...............")
    thread.start()
    

# Wait for all threads to complete
for thread in threads:
    thread.join()
